In [1]:
import pandas as pd
import plotly.express as px

df_cases = pd.read_csv("../time_series_covid19_confirmed_global.csv")
df_deaths = pd.read_csv("../time_series_covid19_deaths_global.csv")
df_recovered = pd.read_csv("../time_series_covid19_recovered_global.csv")

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21


In [2]:
dict_cases = df_cases[df_cases["Country/Region"] == "Germany"].to_dict(orient='dict')
dict_cases.keys()


dict_keys(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20

In [3]:
from dateutil.parser import parse
df_cases_prep = pd.DataFrame(columns=["date", "abs_cases"])
for k, v in dict_cases.items():
    date = ""
    try:
        date = parse(k)
    except:
        continue
    df_cases_prep = df_cases_prep.append({"date": date, "abs_cases": v[133]}, ignore_index=True)

In [24]:

def transform_dataframe(df: pd.DataFrame, c_type: str, country: str = "Germany") -> pd.DataFrame:
    dict_cases = df[df["Country/Region"] == country].to_dict(orient='dict')
    df_prep = pd.DataFrame(columns=["date", f"abs_{c_type}"])
    for k, v in dict_cases.items():
        date = ""
        try:
            date = parse(k)
        except:
            continue
        value_from_dict = 0
        for val in v.values():
            value_from_dict = val
            break
        df_prep = df_prep.append({"date": date, f"abs_{c_type}": value_from_dict}, ignore_index=True)


    for index, row in df_prep.iterrows():
        if index == 0:
            continue
        pre_day = df_prep.at[index - 1, f"abs_{c_type}"]
        cur_day = df_prep.at[index, f"abs_{c_type}"]
        new_c = cur_day - pre_day
        df_prep.at[index, f"new_{c_type}"] = new_c
    return df_prep


In [4]:
df_cases_prep["new_cases"] = 0
for index, row in df_cases_prep.iterrows():
    if index == 0:
        continue
    pre_day = df_cases_prep.at[index - 1, "abs_cases"]
    cur_day = df_cases_prep.at[index, "abs_cases"]
    new_c = cur_day - pre_day
    df_cases_prep.at[index, "new_cases"] = new_c
df_cases_prep.tail(10)

,date,abs_cases,new_cases
358,2021-01-14,2015235,21343
359,2021-01-15,2023828,8593
360,2021-01-16,2038645,14817
361,2021-01-17,2050129,11484
362,2021-01-18,2059382,9253
363,2021-01-19,2071615,12233
364,2021-01-20,2100618,29003
365,2021-01-21,2108895,8277
366,2021-01-22,2125261,16366
367,2021-01-23,2137691,12430


In [14]:
import scipy.signal as sig
px.line(df_cases_prep, x="date", y=sig.savgol_filter(df_cases_prep.new_cases.values, 33, 3))

In [31]:
df_cases_prep = transform_dataframe(df_cases, "Germany", "cases")
df_deaths_prep = transform_dataframe(df_deaths, "Germany", "deaths")
df_recovered_prep = transform_dataframe(df_recovered, "Germany", "recovered")
df_finished = df_cases_prep.merge(df_deaths_prep, how="outer").merge(df_recovered_prep, how="outer").tail(10)

In [ ]:
df_merged = df_cases_prep.merge(df_recovered_prep)
df_merged.head()